# kachaka_apiライブラリ

- kachaka_apiライブラリは、gRPCでの通信を意識することなくカチャカのAPIを使うことができるようにしたものです。

## 非同期ライブラリについて
- このサンプルでは、kachaka_api.aio.KachakaApiClientクラスを用いて、カチャカのAPIをメソッド呼びだしの形で実行していきます。
- 同期のライブラリも用意されています。詳細は、[同期ライブラリ篇](./sample_kachaka_api_client.ipynb)を参照してください。

In [ ]:
# kachaka_apiをimport可能にするため、パスに追加するスクリプトです。サンプル以外の用途では、pip等でinstallしてください。
%run setup_demo_env.py

#### 依存ライブラリのインストール
- デフォルトではmatploblib, numpyはインストールされていません。必要に応じてインストールします。
- pip installの実行後は、JupyterLabのカーネルをリスタートしてください

In [ ]:
%pip install -q matplotlib numpy
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Image, display

## インポートとクライアントの作成
- クライアントを作成します。1度だけでOKです。
- 引数に`<カチャカのIPアドレス:26400>`を指定することで、本ライブラリを外部のPC等で利用することが可能です

In [ ]:
import kachaka_api

client = kachaka_api.aio.KachakaApiClient()

## 各APIの実行

- 以下では、各APIを実行するサンプルを提供します。

### ロボットの個体情報の取得

#### ロボットのシリアル番号の取得

In [ ]:
await client.get_robot_serial_number()

#### カチャカのソフトウェアバージョン情報の取得

In [ ]:
await client.get_robot_version()

### コマンドの実行

#### 目的地の情報一覧を取得
- idは後述の関数で目的地を指定する際に使用します
- poseの単位はロボットの姿勢同様にmおよびradianです

In [ ]:
await client.get_locations()

#### デフォルト目的地のID取得
- デフォルト目的地は、「〇〇を持ってきて」と目的地が省略された場合に使用されます

In [ ]:
await client.get_default_location_id()

#### 家具情報一覧の取得
- idは後述の関数で家具を指定する際に使用します
- poseの単位はロボットの姿勢同様にmおよびradianです

In [ ]:
await client.get_shelves()

#### 家具の移動
- 家具ID, 目的地IDを指定すると、指定した家具を目的地へ運びます。
- IDの一覧は、それぞれ前述のget_shelves(), get_locations()で取得できます。

In [ ]:
await client.move_shelf("S01", "L01")

#### 家具の片付け
- 指定した家具を片付けます。

In [ ]:
await client.return_shelf("S01")

- 引数を省略すると、いま載せている家具を片付けます。

In [ ]:
await client.return_shelf()

#### 家具を置く
- 載せている家具をその場に置きます。

In [ ]:
await client.undock_shelf()

#### 目的地への移動

In [ ]:
await client.move_to_location("L01")

#### 充電ドックに行く

In [ ]:
await client.return_home()

#### 家具を載せる
- カチャカの前にある家具を載せます

In [ ]:
await client.dock_shelf()

#### 音声の発話

In [ ]:
await client.speak("こんにちは、カチャカです")

#### マップ上での座標を指定した移動

In [ ]:
await client.move_to_pose(0.5, 0.0, 0)

#### 実行中のコマンドのキャンセル

In [ ]:
await client.cancel_command()

#### 実行中のコマンドの状態の取得
- コマンドが実行中かどうか調べる

In [ ]:
await client.is_command_running()

- 実行中のコマンドを取得する

In [ ]:
await client.get_running_command()

#### 最後に実行されたコマンドの結果の取得

In [ ]:
await client.get_last_command_result()

#### コマンド実行履歴の取得

In [ ]:
await client.get_history_list()

#### 自動充電のオン・オフを切り替える
- 一定時間速度指令を受け付けないとカチャカは自動で充電ステーションに戻ります。これを自動充電と呼びます

In [ ]:
await client.set_auto_homing_enabled(True)

### 速度指令

#### 手動操作モードへの変更
- 速度を指令してカチャカを動かす場合は手動操縦モードに変更する必要があります。
- 60秒経つと自動でオフになるため、必要に応じて定期的に有効にする必要があります。
- 充電ドック上にいる場合は前に進んで充電ドックから出ます

In [ ]:
await client.set_manual_control_enabled(True)

#### 手動操縦モードの確認

In [ ]:
await client.get_manual_control_enabled()

#### 速度指令
- 指令した値は次の指令値が来るまで、もしくは0.3秒間経つまで保持されます
- 単位はm/s、rad/sです

In [ ]:
for i in range(100):
    await client.set_robot_velocity(0.0, 0.3)

### 各種センサデータ等の取得

#### マップ上の姿勢の取得
- 単位はm, radianです

In [ ]:
await client.get_robot_pose()

#### マップ情報の取得
- マップ画像とマップ座標との対応関係については、[plot_map_lidar.ipynb](./plot_map_lidar.ipynb) を参照

In [ ]:
map = await client.get_png_map()
print(map.name)
print(map.resolution, map.width, map.height)
print(map.origin)
display(Image(data=map.data))

#### LiDARの点群情報の取得
- LiDARは充電ドック上にいる間は停止しており、データが更新されません


In [ ]:
scan = await client.get_ros_laser_scan()

theta = np.linspace(scan.angle_min, scan.angle_max, len(scan.ranges))
dist = np.array(scan.ranges)

# LiDARの点群を表示するサンプル
plt.scatter(dist * np.cos(theta), dist * np.sin(theta))

#### IMU情報の取得

In [ ]:
await client.get_ros_imu()

#### オドメトリの取得

In [ ]:
await client.get_ros_odometry()

### カメラ関連

#### カメラ情報の取得

In [ ]:
await client.get_front_camera_ros_camera_info()

#### フロントカメラ画像の取得

In [ ]:
image = await client.get_front_camera_ros_compressed_image()
display(Image(data=image.data, format="jpeg"))

- また、劣化の無いデータが必要な場合は`get_front_camera_ros_image`で取得する事が可能です。  
- ただし、データ量が増加するため注意が必要です。
- [get_front_camera_ros_image.ipynb](TODO)参照

#### 物体検出結果の取得
- カメラで見えている物体(充電器, 家具, 人)の情報を取得します。
- カメラを起動してから呼び出す必要があります。
- 画像中への描画は[物体検出のサンプル](./get_object_detection.ipynb)参照

In [ ]:
await client.get_object_detection()